In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import SelectPercentile, mutual_info_classif, RFE
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
import json
from scipy.io import arff
import pandas as pd

In [2]:
def auroc(model, data, true_labels):
    prediction = model.predict_proba(data)[:, 1]
    return roc_auc_score(true_labels, prediction)

In [3]:
data = arff.loadarff('data/3year.arff')

with open('data/feature_names.json') as file:
    feature_names = json.load(file)

df = pd.DataFrame(data[0])

In [4]:
y = df['class'].astype(int).astype('category')
X = df.drop('class', axis=1)

In [5]:
X = X.drop('Attr37', axis='columns')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0, shuffle=True, stratify=y)

In [7]:
imputer = SimpleImputer()

imputer.fit(X_train)

X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

In [8]:
resampler = SMOTE(random_state=0, n_jobs=-1)
X_resampled, y_resampled = resampler.fit_resample(X_train, y_train)

/home/integraledelebesgue/Studies/V/ArtificialIntelligence/lab2/venv/lib/python3.11/site-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


## Zadanie dla chętnych

Dokonaj selekcji cech, usuwając 20% najsłabszych cech. Może się tu przydać klasa `SelectPercentile`. Czy Random Forest i LightGBM (bez dostrajania hiperparametrów, dla uproszczenia) wytrenowane bez najsłabszych cech dają lepszy wynik (AUROC lub innej metryki)?

Wykorzystaj po 1 algorytmie z 3 grup algorytmów selekcji cech:
1. Filter methods - mierzymy ważność każdej cechy niezależnie, za pomocą pewnej miary (typowo ze statystyki lub teorii informacji), a potem odrzucamy (filtrujemy) te o najniższej ważności. Są to np. `chi2` i `mutual_info_classif` z pakietu `sklearn.feature_selection`.
2. Embedded methods - klasyfikator sam zwraca ważność cech, jest jego wbudowaną cechą (stąd nazwa). Jest to w szczególności właściwość wszystkich zespołowych klasyfikatorów drzewiastych. Mają po wytrenowaniu atrybut `feature_importances_`.
2. Wrapper methods - algorytmy wykorzystujące w środku używany model (stąd nazwa), mierzące ważność cech za pomocą ich wpływu na jakość klasyfikatora. Jest to np. recursive feature elimination (klasa `RFE`). W tym algorytmie trenujemy klasyfikator na wszystkich cechach, wyrzucamy najsłabszą, trenujemy znowu i tak dalej.

Typowo metody filter są najszybsze, ale dają najsłabszy wynik, natomiast metody wrapper są najwolniejsze i dają najlepszy wynik. Metody embedded są gdzieś pośrodku.

Dla zainteresowanych, inne znane i bardzo dobre algorytmy:
- Relief (filter method) oraz warianty, szczególnie ReliefF, SURF i MultiSURF (biblioteka `ReBATE`): [Wikipedia](https://en.wikipedia.org/wiki/Relief_(feature_selection)), [artykuł "Benchmarking Relief-Based Feature Selection Methods"](https://www.researchgate.net/publication/321307194_Benchmarking_Relief-Based_Feature_Selection_Methods)
- Boruta (wrapper method), stworzony na Uniwersytecie Warszawskim, łączący Random Forest oraz testy statystyczne (biblioteka `boruta_py`): [link 1](https://towardsdatascience.com/boruta-explained-the-way-i-wish-someone-explained-it-to-me-4489d70e154a), [link 2](https://danielhomola.com/feature%20selection/phd/borutapy-an-all-relevant-feature-selection-method/)

In [9]:
def select_and_test(model, selector, X_train=X_resampled, X_test=X_test, y_train=y_resampled, y_test=y_test):
    selector.fit(X_train, y_train)

    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)

    model.fit(X_train_selected, y_train)

    return auroc(model, X_test_selected, y_test)

In [10]:
models = {
    'Decision Tree': DecisionTreeClassifier(criterion='entropy', splitter='best', random_state=0),
    'Random Forest': RandomForestClassifier(n_estimators=500, criterion='entropy', n_jobs=-1, random_state=0),
    'LGBM': LGBMClassifier(random_state=0, n_jobs=-1, verbose=-100)
}

In [11]:
def test_selector(selector, models=models):
    print('AUROC score:')
    for name, model in models.items():
        score = select_and_test(model, selector)
        print(f'{name}: {score}')

In [12]:
filter_selector = SelectPercentile(
    mutual_info_classif,
    percentile=80,    
)

test_selector(filter_selector)

AUROC score:
Decision Tree: 0.6736207463376852
Random Forest: 0.8902327127783072
LGBM: 0.910447650304891
